In [1]:
# This Python 3 environment comes with ma
# ny helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# Install required libraries for this notebook
!pip install -U google-generativeai python-dotenv pymupdf



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


# TaskFlow: An Autonomous Research & Execution Agent 🧠⚙️

This project demonstrates a GenAI-based autonomous agent that:
- Understands complex research queries
- Breaks them into subtasks
- Retrieves and summarizes relevant info using LLM
- Outputs structured research reports (in JSON/Markdown)

Models used: `gemini-1.5-pro`, `text-embedding-004`


In [3]:
import google.generativeai as genai
import json

# Show the installed google-generativeai version
genai.__version__


'0.8.5'

In [4]:
import google.generativeai as genai
from dotenv import load_dotenv
import os

load_dotenv()
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

for m in genai.list_models():
    print(m.name)


models/embedding-gecko-001
models/gemini-2.5-flash
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-exp-1206
models/gemini-2.5-flash-preview-tts
models/gemini-2.5-pro-preview-tts
models/gemma-3-1b-it
models/gemma-3-4b-it
models/gemma-3-12b-it
models/gemma-3-27b-it
models/gemma-3n-e4b-it
models/gemma-3n-e2b-it
models/gemini-flash-latest
models/gemini-flash-lite-latest
models/gemini-pro-latest
models/gemini-2.5-flash-lite
models/gemini-2.5-flash-image-preview
models/gemini-2.5-flash-image
models/gemini-2.5-flash-preview-09-2025
models/gemini-2.5-flash-lite-preview-09-2025
models/gemini-3-pro-preview
models/gemini-3-pro-image-preview
models/nano-banana-pro-preview
models/gemini-robotics-er-1.5-preview
models/gemini-2.5-computer-use-preview

In [5]:
import google.generativeai as genai
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=api_key)

model = genai.GenerativeModel("gemini-2.5-flash")

response = model.generate_content("Hello! This is a test.")
print(response.text)


Hello! I'm ready. How can I assist you with this test?


In [6]:
from google.api_core import retry

def is_retriable(exception):
    return isinstance(exception, Exception) and (
        "429" in str(exception) or "503" in str(exception)
    )

retry_wrapper = retry.Retry(predicate=is_retriable)

def generate_with_retry(model, *args, **kwargs):
    return retry_wrapper(model.generate_content)(*args, **kwargs)


### Using the gemini-1.5-pro, and answering a prompt 

In [7]:
import google.generativeai as genai
from dotenv import load_dotenv
import os

load_dotenv()
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

short_config = {"max_output_tokens": 2000}

model = genai.GenerativeModel("gemini-2.5-flash")

response = model.generate_content(
    "Find best open-source LLMs with license info in 200 words",
    generation_config=short_config,
)

print(response.text)


Here's a breakdown of some of the best open-source LLMs with their license information:

1.  **Meta Llama 2 & 3:**
    *   **Strengths:** Leading in performance and adoption, available in various sizes (7B, 13B, 70B, 8B, 70B, 400B+). Excellent for a wide range of tasks.
    *   **License:** Meta Llama Community Licenses (Llama 2 Community License, Meta Llama 3 Community License). These generally permit commercial use, but include revenue-based restrictions for very large enterprises (over $700M USD annual revenue) requiring special licensing.

2.  **Mistral AI Models (Mistral 7B, Mixtral 8x7B):**
    *   **Strengths:** Exceptional performance-to-size ratio. Mixtral 8x7B is a Mixture-of-Experts (MoE) model offering high quality with efficient inference.
    *   **License:** Apache 2.0. This is a highly permissive open-source license, allowing extensive commercial use, modification, and distribution with attribution.

3.  **Google Gemma:**
    *   **Strengths:** Lightweight, open models 

### Responding some user queries and returning the response in json format

In [8]:
import google.generativeai as genai
import typing_extensions as typing
import json
from dotenv import load_dotenv
import os

load_dotenv()
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

class TaskPlan(typing.TypedDict):
    tasks: list[str]

generation_config = {
    "temperature": 0.2,
    "response_mime_type": "application/json",
    "response_schema": TaskPlan,
}

model = genai.GenerativeModel("gemini-2.5-flash")

def plan_tasks(topic: str):
    response = model.generate_content(
        f"Break into 3–5 research tasks about: '{topic}'. "
        "Return JSON with a single key 'tasks' containing an array of task strings.",
        generation_config=generation_config,
    )
    return response.text

user_query = "Role of AI in sustainable development"  # queries
tasks = plan_tasks(user_query)

parsed_tasks = json.loads(tasks)
print(json.dumps(parsed_tasks, indent=2))


{
  "tasks": [
    "Identify current applications and case studies of AI in various sustainable development sectors (e.g., renewable energy, smart agriculture, climate modeling, waste management).",
    "Analyze the potential benefits and contributions of AI technologies towards achieving specific United Nations Sustainable Development Goals (SDGs).",
    "Examine the challenges, risks, and ethical considerations associated with deploying AI for sustainable development, including energy consumption, data privacy, bias, and job displacement.",
    "Investigate policy frameworks, governance models, and international collaborations required to ensure responsible and equitable AI development and deployment for sustainability.",
    "Explore future trends and emerging AI technologies that could further accelerate progress in sustainable development and address global environmental and social challenges."
  ]
}


# Agent Loop
[ User query ] → [ Task planner ] → [ Subtasks ] → [ Execute each task ]

- It decomposes each tasks into subtasks
- Then executes each subtasks using GenAI


In [9]:
import google.generativeai as genai
model = genai.GenerativeModel(model_name="gemini-2.5-flash") # using a different model as gemini 1.5 pro has limitations on rpm



for i, task in enumerate(parsed_tasks['tasks'], 1):
    print(f"\n🔹 Task {i}: {task}")
    
    # Ask Gemini to answer each task
    response = model.generate_content(f"Write a short paragraph on: {task}")
    
    print("📝 Response:")
    print(response.text)



🔹 Task 1: Identify current applications and case studies of AI in various sustainable development sectors (e.g., renewable energy, smart agriculture, climate modeling, waste management).


ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 5, model: gemini-2.5-flash
Please retry in 44.210378852s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 5
}
, retry_delay {
  seconds: 44
}
]

# Document Understanding

Reading documents and extracting text from it

In [ ]:
#!pip install pymupdf

import fitz  # PyMuPDF

def extract_text_from_pdf(path):
    doc = fitz.open(path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

raw_text = extract_text_from_pdf('H:\GATE SYLLABUS.pdf')


<>:12: SyntaxWarning: invalid escape sequence '\G'
<>:12: SyntaxWarning: invalid escape sequence '\G'
C:\Users\hp\AppData\Local\Temp\ipykernel_22608\1839195758.py:12: SyntaxWarning: invalid escape sequence '\G'
  raw_text = extract_text_from_pdf('H:\GATE SYLLABUS.pdf')



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Breaking the long pdf into chunks so that it is made easier for the model to understand and give output efficiently with minimal errors.

In [ ]:
def chunk_text(text, chunk_size=100, overlap=50):
    chunks = []
    start = 0
    while start < len(text):
        end = min(start + chunk_size, len(text))
        chunks.append(text[start:end])
        start += chunk_size - overlap
    return chunks

chunks = chunk_text(raw_text)

### Embed the chunks

Embedding the chunks into vectors for semantic understanding.

using embedding model for embedding the chunks.

In [ ]:
# Batching the chunks as the request is limited to 100
import google.generativeai as genai
import numpy as np

def batch_chunks(chunks, batch_size=100):
    for i in range(0, len(chunks), batch_size):
        yield chunks[i:i + batch_size]

all_embeddings = []

for batch in batch_chunks(chunks, batch_size=100):
    response = genai.embed_content(
        model="text-embedding-004",
        content=batch,
        task_type="semantic_similarity",
    )

    # Case 1: batch of many → response["embeddings"]
    if "embeddings" in response:
        vectors = [item["embedding"] for item in response["embeddings"]]

    # Case 2: batch of one → response["embedding"]
    elif "embedding" in response:
        vectors = [response["embedding"]]

    else:
        raise ValueError("Unexpected embedding response format:", response)

    # FIX: Convert token-level embeddings (N,768) to sentence-level (768,)
    processed_vectors = []
    for v in vectors:
        v = np.array(v)
        if v.ndim == 2:            # shape (N, 768)
            v = v.mean(axis=0)     # convert to (768,)
        processed_vectors.append(v.tolist())

    all_embeddings.extend(processed_vectors)


The embeddedd vectors are now being stored in chunk_db

In [ ]:
# Assuming all_embeddings and chunks are in order
chunk_db = [
    {"text": chunk, "embedding": embedding}
    for chunk, embedding in zip(chunks, all_embeddings)
]


Also embedding the query into vectors so that the model can match these embedding with the relevant content from the pdf.

In [ ]:
query = "what are the important topics"

In [ ]:
import google.generativeai as genai

query_embedding_response = genai.embed_content(
    model="text-embedding-004",
    content=query,
    task_type="semantic_similarity",
)

# Single embedding vector for the query
query_embedding = query_embedding_response.get("embedding") or query_embedding_response.get("embeddings", [None])[0]


Calculating the similarity scores using cosine similarity and finding the top k similar chunks

In [ ]:
from numpy import dot
from numpy.linalg import norm
import numpy as np

def cosine_similarity(a, b):
    return dot(a, b) / (norm(a) * norm(b))

top_k = 3

scored_chunks = [
    (cosine_similarity(np.array(query_embedding), np.array(chunk["embedding"])), chunk["text"])
    for chunk in chunk_db
]

ranked_chunks = sorted(scored_chunks, key=lambda x: x[0], reverse=True)
top_chunks = [text for _, text in ranked_chunks[:top_k]]


Giving a rag prompt that specifies what to do to the model

and generating the output using the gemini model.


In [ ]:
print(type(chunks[0]))
print(chunks[0][:2000])


<class 'str'>
 
CS Computer Science and Information Technology 
Section 1: Engineering Mathematics 
Discrete Mathe


In [ ]:
rag_prompt = (
    "Answer the following question using the provided context.\n\n"
    f"Context:\n{''.join(top_chunks)}\n\n"
    f"Question:\n{query}"
)

import google.generativeai as genai

response = genai.GenerativeModel("gemini-2.5-flash").generate_content(rag_prompt)

print("🔍 Final Answer:\n", response.text)


🔍 Final Answer:
 Based on the provided context, the important topic mentioned is **Discrete Mathematics**.

The context does not provide a further breakdown of specific important topics *within* Discrete Mathematics.
